# Lab5

In [3]:
import sklearn.datasets
import scipy as sc

def load_iris():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    return D, L


In [4]:
import numpy

def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0)
    numpy.random.seed(seed)
    idx = numpy.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]
    DTR = D[:, idxTrain]
    DTE = D[:, idxTest]
    LTR = L[idxTrain]
    LTE = L[idxTest]
    return (DTR, LTR), (DTE, LTE)


In [5]:
D, L = load_iris()
(DTR, LTR), (DTE, LTE) = split_db_2to1(D, L)

In [62]:
def vcol(vector, shape0):
    # Auxiliary function to transform 1-dim vectors to column vectors.
    return vector.reshape(shape0, 1)


def vrow(vector, shape1):
    # Auxiliary function to transform 1-dim vecotrs to row vectors.
    return vector.reshape(1, shape1)

In [63]:
def logpdf_GAU_ND(x, mu, sigma):
    return -(x.shape[0]/2)*numpy.log(2*numpy.pi)-(1/2)*(numpy.linalg.slogdet(sigma)[1])-(1/2)*((numpy.dot((x-mu).T, numpy.linalg.inv(sigma))).T*(x-mu)).sum(axis=0)

In [64]:
def computeScoreMatrix(D, mu0, sigma0, mu1, sigma1, mu2, sigma2, callback):
    S = numpy.array([callback(D, mu0, sigma0), callback(
        D, mu1, sigma1), callback(D, mu2, sigma2)])
    return S

In [65]:
def computeMLestimates(D, L):
    # Compute classes means over columns of the dataset matrix
    mu0 = D[:, L == 0].mean(axis=1)
    mu1 = D[:, L == 1].mean(axis=1)
    mu2 = D[:, L == 2].mean(axis=1)
    # Reshape all of them as 4x1 column vectors
    mu0 = vcol(mu0, mu0.size)
    mu1 = vcol(mu1, mu1.size)
    mu2 = vcol(mu2, mu2.size)
    # Count number of elements in each class
    n0 = D[:, L == 0].shape[1]
    n1 = D[:, L == 1].shape[1]
    n2 = D[:, L == 2].shape[1]
    # Subtract classes means from classes datasets with broadcasting
    DC0 = D[:, L == 0]-mu0
    DC1 = D[:, L == 1]-mu1
    DC2 = D[:, L == 2]-mu2
    # Compute classes covariance matrices
    sigma0 = (1/n0)*(numpy.dot(DC0, DC0.T))
    sigma1 = (1/n1)*(numpy.dot(DC1, DC1.T))
    sigma2 = (1/n2)*(numpy.dot(DC2, DC2.T))
    return (mu0, sigma0), (mu1, sigma1), (mu2, sigma2)

In [66]:
def multivariateGaussianClassifier(DTR, LTR, DEV, LEV):
    # Compute estimates for model parameters (empirical mean
    # and covariance matrix of each class). This is the training phase.
    (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
    # Now we have the estimated model parameters and we can turn our attention towards
    # inference for a test sample x of the evaluation set. The final goal is to
    # compute class posterior probabilities, but we split the process in three stages.

    # 1) Compute, for each test sample, the MVG log-density.
    # We can proceed as seen in lab 04 and we can store class-conditional
    # probabilities (the computed log-densities) in a score matrix logS. logS[i, j]
    # should be the class conditional probability for sample j given class i.
    logS = computeScoreMatrix(DTE, mu0, sigma0, mu1,
                          sigma1, mu2, sigma2, logpdf_GAU_ND)
    # 2) Compute the matrix of joint log-distribution probabilities logSJoint for
    # samples and classes combining the score matrix with prior information.
    # We assume that the three classes have the same
    # prior probability P(c) = 1/3. logSJoints requires adding each row of
    # logS to the logarithm of the prior probability of the corresponding class.
    priorLogProbabilities = vcol(
        numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
    logSJoint = logS+priorLogProbabilities  # 3x50
    # 3) We can finally compute class posterior probabilities. But we need to compute
    # the marginal log-density. We can use scipy.special.logsumexp(logSJoint, axis=0)
    marginalLogDensities = vrow(
        sc.special.logsumexp(logSJoint, axis=0), 50)  # 1x50
    # Now we can compute the array of class log-posterior probabilities logSPost.
    logSPost = logSJoint-marginalLogDensities
    # The predicted label is obtained as the class that has maximum posterior
    # probability, in our 3x50 logSPost matrix. This needs to be done for each sample.
    # We can use argmax with axis=0 on the logSPost matrix. It will return an
    # array whose values are the indices (in our case 0, 1, 2) of the maximum
    # values along the specified axis. (So, for us is the maximum of each column)
    predictedLabels = logSPost.argmax(axis=0)
    # We can now compute an array of booleans corresponding to whether predicted
    # and real labels are equal or not. Then, summing all the elements of a
    # boolean array gives the number of elements that are True.
    numberOfCorrectPredictions = numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/LEV.size*100
    errorRate = 100-accuracy
    print("Accuracy of the MVG classifier when working with log-densities: %.2f %%" % (accuracy))
    print("Error rate: %.2f %%" % (errorRate))
    return

In [67]:
def naiveBayesGaussianClassifier(DTR, LTR, DEV, LEV):
    # Compute estimates for model parameters (empirical mean
    # and covariance matrix of each class). This is the training phase.
    (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
    # But for the Naive Bayes version of MVG the covariance matrices are diagonal!
    # The ML solution for the mean parameters is the same, while the ML solution
    # for the covariance matrices is different. Since the number of feature is
    # small, we can adapt the MVG code by simply zeroing the out-of-diagonal
    # elements of the MVG ML solution, keeping them as matrices. This can be
    # done, for example, multiplying element-wise the MVG ML solution with the
    # identity matrix. The rest of the code is the same as MVGlogdensities.
    # This procedure (keeping the matrix, etc.) is not advisable when we work
    # on large dimensional data, in those cases it may be better to implement
    # ad-hoc functions to work directly with just the diagonal of the covariance
    # matrices.
    (sigma0, sigma1, sigma2) = (sigma0*numpy.identity(sigma0.shape[0]), sigma1*numpy.identity(
        sigma1.shape[0]), sigma2*numpy.identity(sigma2.shape[0]))
    # Now we have the estimated model parameters and we can turn our attention towards
    # inference for a test sample x of the evaluation set. The final goal is to
    # compute class posterior probabilities, but we split the process in three stages.

    # 1) Compute, for each test sample, the MVG log-density.
    # We can proceed as seen in lab 04 and we can store class-conditional
    # probabilities (the computed log-densities) in a score matrix logS. logS[i, j]
    # should be the class conditional probability for sample j given class i.
    logS = computeScoreMatrix(DTE, mu0, sigma0, mu1,
                          sigma1, mu2, sigma2, logpdf_GAU_ND)
    # 2) Compute the matrix of joint log-distribution probabilities logSJoint for
    # samples and classes combining the score matrix with prior information.
    # We assume that the three classes have the same
    # prior probability P(c) = 1/3. logSJoints requires adding each row of
    # logS to the logarithm of the prior probability of the corresponding class.
    priorLogProbabilities = vcol(
        numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
    logSJoint = logS+priorLogProbabilities  # 3x50
    # 3) We can finally compute class posterior probabilities. But we need to compute
    # the marginal log-density. We can use scipy.special.logsumexp(logSJoint, axis=0)
    marginalLogDensities = vrow(
        sc.special.logsumexp(logSJoint, axis=0), 50)  # 1x50
    # Now we can compute the array of class log-posterior probabilities logSPost.
    logSPost = logSJoint-marginalLogDensities
    # The predicted label is obtained as the class that has maximum posterior
    # probability, in our 3x50 logSPost matrix. This needs to be done for each sample.
    # We can use argmax with axis=0 on the logSPost matrix. It will return an
    # array whose values are the indices (in our case 0, 1, 2) of the maximum
    # values along the specified axis. (So, for us is the maximum of each column)
    predictedLabels = logSPost.argmax(axis=0)
    # We can now compute an array of booleans corresponding to whether predicted
    # and real labels are equal or not. Then, summing all the elements of a
    # boolean array gives the number of elements that are True.
    numberOfCorrectPredictions = numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/LEV.size*100
    errorRate = 100-accuracy
    print("Accuracy of the naive bayes MVG classifier when working with log-densities: %.2f %%" % (accuracy))
    print("Error rate: %.2f %%" % (errorRate))
    return

In [68]:
def tiedGaussianClassifier(DTR, LTR, DEV, LEV):
    # Compute estimates for model parameters (empirical mean
    # and covariance matrix of each class). This is the training phase.
    (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
    # But for the tied covariance version of the classifier the class covariance
    # matrices are tied, this mean that sigmai=sigma, they're all the same.
    # We have seen that the ML solution for the class means is again the same.
    # The ML solution for the covariance matrix, instead, is given by the empirical
    # within-class covariance matrix. We already computed it when we implemented
    # LDA, alternatively (and I will do so in the following) we can compute it
    # from the covariance matrices sigma0, sigma1 and sigma2:
    sigma = (1/DTR.shape[1])*((LTR == 0).sum()*sigma0 +
                              (LTR == 1).sum()*sigma1+(LTR == 2).sum()*sigma2)
    # Now we have the estimated model parameters and we can turn our attention towards
    # inference for a test sample x of the evaluation set. The final goal is to
    # compute class posterior probabilities, but we split the process in three stages.

    # 1) Compute, for each test sample, the MVG log-density.
    # We can proceed as seen in lab 04 and we can store class-conditional
    # probabilities (the computed log-densities) in a score matrix logS. logS[i, j]
    # should be the class conditional probability for sample j given class i.
    logS = computeScoreMatrix(DTE, mu0, sigma, mu1,
                          sigma, mu2, sigma, logpdf_GAU_ND)
    # 2) Compute the matrix of joint log-distribution probabilities logSJoint for
    # samples and classes combining the score matrix with prior information.
    # We assume that the three classes have the same
    # prior probability P(c) = 1/3. logSJoints requires adding each row of
    # logS to the logarithm of the prior probability of the corresponding class.
    priorLogProbabilities = vcol(
        numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
    logSJoint = logS+priorLogProbabilities  # 3x50
    # 3) We can finally compute class posterior probabilities. But we need to compute
    # the marginal log-density. We can use scipy.special.logsumexp(logSJoint, axis=0)
    marginalLogDensities = vrow(
        sc.special.logsumexp(logSJoint, axis=0), 50)  # 1x50
    # Now we can compute the array of class log-posterior probabilities logSPost.
    logSPost = logSJoint-marginalLogDensities
    # The predicted label is obtained as the class that has maximum posterior
    # probability, in our 3x50 logSPost matrix. This needs to be done for each sample.
    # We can use argmax with axis=0 on the logSPost matrix. It will return an
    # array whose values are the indices (in our case 0, 1, 2) of the maximum
    # values along the specified axis. (So, for us is the maximum of each column)
    predictedLabels = logSPost.argmax(axis=0)
    # We can now compute an array of booleans corresponding to whether predicted
    # and real labels are equal or not. Then, summing all the elements of a
    # boolean array gives the number of elements that are True.
    numberOfCorrectPredictions = numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/LEV.size*100
    errorRate = 100-accuracy
    print("Accuracy of the tied MVG classifier when working with log-densities: %.2f %%" % (accuracy))
    print("Error rate: %.2f %%" % (errorRate))
    return

In [69]:
multivariateGaussianClassifier(DTR, LTR, DTE, LTE)
naiveBayesGaussianClassifier(DTR, LTR, DTE, LTE)
tiedGaussianClassifier(DTR, LTR, DTE, LTE)

Accuracy of the MVG classifier when working with log-densities: 96.00 %
Error rate: 4.00 %
Accuracy of the naive bayes MVG classifier when working with log-densities: 96.00 %
Error rate: 4.00 %
Accuracy of the tied MVG classifier when working with log-densities: 98.00 %
Error rate: 2.00 %


# K-fold approch

In [86]:
def kfoldCrossValidationMVG(D, L):
    # Same procedure of MVGlogdensities but with leave-one-out approach:
    # number of folds is equal to the number of samples, we iterate over all samples
    # and each time we use one of them as evaluation set and the remaining 149
    # as training set.
    # Then we compute the predicted lable and count how many times it is correct.
    numberOfCorrectPredictions = 0
    for i in range(D.shape[1]):  # i=0...149
        # Each time we need to delete the i-th column (sample) from the training set
        # while in the evaluation set we should keep only the i-th column.
        # vcol is necessary to exploit broadcasting
        (DTR, LTR), (DEV, LEV) = (numpy.delete(D, i, 1),
                                  numpy.delete(L, i)), (vcol(D[:, i], 4), L[i])
        (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
        logS = computeScoreMatrix(DEV, mu0, sigma0, mu1,
                                  sigma1, mu2, sigma2, logpdf_GAU_ND)
        priorLogProbabilities = vcol(
            numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
        logSJoint = logS+priorLogProbabilities  # 3x1
        marginalLogDensities = vrow(
            sc.special.logsumexp(logSJoint, axis=0), 1)  # 1x1
        # We need to keep it as 1x1 to exploit broadcasting
        logSPost = logSJoint-marginalLogDensities
        predictedLabels = logSPost.argmax(axis=0)
        # Add the result of prediction to the proper variable
        numberOfCorrectPredictions += numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/L.size * \
        100  # over all samples, that are 150
    errorRate = 100-accuracy
    print("MVG classifier: accuracy : %.2f %%" % (accuracy))
    print("MVG classifier: error rate: %.2f %%" % (errorRate))
    return

In [87]:
def kfoldCrossValidationNaiveBayesMVG(D, L):
    # Same procedure of MVGlogdensities but with leave-one-out approach:
    # number of folds is equal to the number of samples, we iterate over all samples
    # and each time we use one of them as evaluation set and the remaining 149
    # as training set.
    # Then we compute the predicted lable and count how many times it is correct.
    numberOfCorrectPredictions = 0
    for i in range(D.shape[1]):  # i=0...149
        # Each time we need to delete the i-th column (sample) from the training set
        # while in the evaluation set we should keep only the i-th column.
        # vcol is necessary to exploit broadcasting
        (DTR, LTR), (DEV, LEV) = (numpy.delete(D, i, 1),
                                  numpy.delete(L, i)), (vcol(D[:, i], 4), L[i])
        (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
        (sigma0, sigma1, sigma2) = (sigma0*numpy.identity(sigma0.shape[0]), sigma1*numpy.identity(
        sigma1.shape[0]), sigma2*numpy.identity(sigma2.shape[0]))
        logS = computeScoreMatrix(DEV, mu0, sigma0, mu1,
                                  sigma1, mu2, sigma2, logpdf_GAU_ND)
        priorLogProbabilities = vcol(
            numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
        logSJoint = logS+priorLogProbabilities  # 3x1
        marginalLogDensities = vrow(
            sc.special.logsumexp(logSJoint, axis=0), 1)  # 1x1
        # We need to keep it as 1x1 to exploit broadcasting
        logSPost = logSJoint-marginalLogDensities
        predictedLabels = logSPost.argmax(axis=0)
        # Add the result of prediction to the proper variable
        numberOfCorrectPredictions += numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/L.size * \
        100  # over all samples, that are 150
    errorRate = 100-accuracy
    print("MVG naive classifier: accuracy : %.2f %%" % (accuracy))
    print("MVG naive classifier: error rate: %.2f %%" % (errorRate))
    return

In [88]:
def kfoldCrossValidationTiedMVG(D, L):
    # Same procedure of MVGlogdensities but with leave-one-out approach:
    # number of folds is equal to the number of samples, we iterate over all samples
    # and each time we use one of them as evaluation set and the remaining 149
    # as training set.
    # Then we compute the predicted lable and count how many times it is correct.
    numberOfCorrectPredictions = 0
    for i in range(D.shape[1]):  # i=0...149
        # Each time we need to delete the i-th column (sample) from the training set
        # while in the evaluation set we should keep only the i-th column.
        # vcol is necessary to exploit broadcasting
        (DTR, LTR), (DEV, LEV) = (numpy.delete(D, i, 1),
                                  numpy.delete(L, i)), (vcol(D[:, i], 4), L[i])
        (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
        sigma = (1/DTR.shape[1])*((LTR == 0).sum()*sigma0 +
                              (LTR == 1).sum()*sigma1+(LTR == 2).sum()*sigma2)
        logS = computeScoreMatrix(DEV, mu0, sigma, mu1,
                                  sigma, mu2, sigma, logpdf_GAU_ND)
        priorLogProbabilities = vcol(
            numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
        logSJoint = logS+priorLogProbabilities  # 3x1
        marginalLogDensities = vrow(
            sc.special.logsumexp(logSJoint, axis=0), 1)  # 1x1
        # We need to keep it as 1x1 to exploit broadcasting
        logSPost = logSJoint-marginalLogDensities
        predictedLabels = logSPost.argmax(axis=0)
        # Add the result of prediction to the proper variable
        numberOfCorrectPredictions += numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/L.size * \
        100  # over all samples, that are 150
    errorRate = 100-accuracy
    print("MVG tied classifier: accuracy : %.2f %%" % (accuracy))
    print("MVG tied classifier: error rate: %.2f %%" % (errorRate))
    return

In [89]:
def kfoldCrossValidationTiedNaiveBayesMVG(D, L):
    # Same procedure of MVGlogdensities but with leave-one-out approach:
    # number of folds is equal to the number of samples, we iterate over all samples
    # and each time we use one of them as evaluation set and the remaining 149
    # as training set.
    # Then we compute the predicted lable and count how many times it is correct.
    numberOfCorrectPredictions = 0
    for i in range(D.shape[1]):  # i=0...149
        # Each time we need to delete the i-th column (sample) from the training set
        # while in the evaluation set we should keep only the i-th column.
        # vcol is necessary to exploit broadcasting
        (DTR, LTR), (DEV, LEV) = (numpy.delete(D, i, 1),
                                  numpy.delete(L, i)), (vcol(D[:, i], 4), L[i])
        (mu0, sigma0), (mu1, sigma1), (mu2, sigma2) = computeMLestimates(DTR, LTR)
        sigma = (1/DTR.shape[1])*((LTR == 0).sum()*sigma0 +
                              (LTR == 1).sum()*sigma1+(LTR == 2).sum()*sigma2)
        sigma = sigma*numpy.identity(sigma.shape[0])
        logS = computeScoreMatrix(DEV, mu0, sigma, mu1,
                                  sigma, mu2, sigma, logpdf_GAU_ND)
        priorLogProbabilities = vcol(
            numpy.array([numpy.log(1/3), numpy.log(1/3), numpy.log(1/3)]), 3)
        logSJoint = logS+priorLogProbabilities  # 3x1
        marginalLogDensities = vrow(
            sc.special.logsumexp(logSJoint, axis=0), 1)  # 1x1
        # We need to keep it as 1x1 to exploit broadcasting
        logSPost = logSJoint-marginalLogDensities
        predictedLabels = logSPost.argmax(axis=0)
        # Add the result of prediction to the proper variable
        numberOfCorrectPredictions += numpy.array(predictedLabels == LEV).sum()
    # Now we can compute percentage values for accuracy and error rate.
    accuracy = numberOfCorrectPredictions/L.size * \
        100  # over all samples, that are 150
    errorRate = 100-accuracy
    print("MVG tied naive classifier: accuracy : %.2f %%" % (accuracy))
    print("MVG tied naive classifier: error rate: %.2f %%" % (errorRate))
    return

In [91]:
kfoldCrossValidationMVG(D, L)
kfoldCrossValidationNaiveBayesMVG(D, L)
kfoldCrossValidationTiedMVG(D, L)
kfoldCrossValidationTiedNaiveBayesMVG(D, L)

MVG classifier: accuracy : 97.33 %
MVG classifier: error rate: 2.67 %
MVG naive classifier: accuracy : 95.33 %
MVG naive classifier: error rate: 4.67 %
MVG tied classifier: accuracy : 98.00 %
MVG tied classifier: error rate: 2.00 %
MVG tied naive classifier: accuracy : 96.00 %
MVG tied naive classifier: error rate: 4.00 %


In [32]:
import numpy
idx = numpy.random.permutation(2)
idx

array([1, 0])

In [34]:
K = 3
i = 0
n_sample_fold = int(2)
#D[:,idx[(i*n_sample_fold): ((i+1)*(n_sample_fold))]]
D[:, idx]


array([[4.9, 5.1],
       [3. , 3.5],
       [1.4, 1.4],
       [0.2, 0.2]])

In [48]:
fold_sizes = numpy.full(4, 11 // 4, dtype=int)
fold_sizes[: 11 % 4] 
#fold_sizes

array([2, 2, 2])

In [56]:
import numpy as np
y = np.asarray(L)
L

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [57]:
_, y_idx, y_inv = np.unique(y, return_index=True, return_inverse=True)


In [58]:
y_idx

array([  0,  50, 100])

In [61]:
_, class_perm = np.unique(y_idx, return_inverse=True)
class_perm

array([0, 1, 2])

In [63]:
y_encoded = class_perm[y_inv]
y_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [64]:
n_classes = len(y_idx)
y_counts = np.bincount(y_encoded)
y_counts

array([50, 50, 50])

In [66]:
min_groups = np.min(y_counts)
min_groups

50

In [75]:
y_order = np.sort(y_encoded)
allocation = np.asarray(
    [
        np.bincount(y_order[i :: 3], minlength=n_classes)
        for i in range(3)
    ]
)

In [70]:
y_order[i :: 3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2])

In [73]:
test_folds = np.empty(len(y), dtype="i")


In [85]:
for k in range(3):
    # since the kth column of allocation stores the number of samples
    # of class k in each test set, this generates blocks of fold
    # indices corresponding to the allocation for class k.
    folds_for_class = np.arange(3).repeat(allocation[:, k])
    test_folds[y_encoded == k] = folds_for_class

In [81]:
folds_for_class = np.arange(3).repeat(allocation[:, 1])
folds_for_class


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2])

In [80]:
test_folds[y_encoded == k] = folds_for_class

array([0, 1, 2])

In [86]:
test_folds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)